# Project Desription
#### PROBLEM STATEMENT <i>(according to the data provider on Kaggle)</i>     

"A response model can provide a significant boost to the efficiency of a marketing campaign by increasing responses or reducing expenses. The objective is to predict who will respond to an offer for a product or service."  

##### GOAL OF THE PROJECT    

We are required to model and train a predictive model which allows the company to maximize the profit of the next marketing campaign, leveraging the resulting key-drivers of the model.  

##### DATA SOURCE  

* FROM Kaggle - https://www.kaggle.com/rodsaldanha/arketing-campaign


**Okay so prima facie, lets import some of the libraries we will be needing for this project.**  
(even if we are missing out on some library, we can always import it later on in the project.)

In [0]:
import sys

# library to handle vectorized data 
import numpy as np 
# library for data analsysis and man

import pandas as pd 

# for visualisations
import seaborn as sns

%matplotlib inline 
import matplotlib as mpl
import matplotlib.pyplot as plt


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Loading the data.